In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

from random import *
import itertools 
from scipy.optimize import minimize

In [ ]:
input_df = pd.read_pickle('dlight_data.pkl')
data = input_df
pd.set_option("display.max.columns", None)

In [ ]:
session = data['session'].values.tolist() 
switch = [i for i in range(1,len(session)) if session[i]!=session[i-1] ]
switch.append(len(session))
block = list(np.diff(switch))
block = [451] + block

In [ ]:
name = [i for i in range(len(block))]

In [ ]:
temp = []
for i, x in enumerate(block):
    t = [name[i] for a in range(block[i])]
    temp.append(t)
flat_list = [item for sublist in temp for item in sublist]

In [ ]:
data['mouse'] = flat_list

In [ ]:
def softmax(beta, value):   
    num = np.exp(value * beta)
    den = np.exp(value * beta).sum()    
    return num / den

In [ ]:
class parameter_fit(object):
    def __init__(self, df, bounds = ((0,1), (0,5)), guess = [0.1,1]):
        self.df = df
        self.bounds = bounds # range for alpha and beta
        self.guess = guess  # guess to aid scipy minimize
        
    def negative_log_likelihood(self, parameter):
        df = self.df
        alpha = parameter[0] # defining separate parameters
        beta = parameter[1]
        value = 0.5 * np.ones(2) # ensuring computes values for both levers/choices. Value starts at 0.5
        
        choices, rewards = df['right_or_left'].values.astype(int), df['reward'].values.astype(int)
        prob_log = 0
        session = df['session'].values.tolist()
        switch = [i for i in range(1,len(session)) if session[i]!=session[i-1] ]
        
        
        for choice, reward in zip(choices, rewards):
            if choice not in switch:
                value[choice] += alpha * (reward - value[choice])  
            else:
                value = 0.5
            prob_log += np.log(softmax(value, beta)[choice])

        return -prob_log

    def minimisation(self):
        bounds = (self.bounds)
        mini = minimize(self.negative_log_likelihood, self.guess,
                     method='L-BFGS-B',
                     bounds=bounds)  # method is optimisation algorithm,
        return mini

    def data_fit(self):
        data_fit = pd.DataFrame()
        for mice in self.df['mouse'].unique().tolist():
            df1 = self.df[self.df['mouse'] == mice]
            df1 = df1.reset_index(drop = True)
            mouse = df1['mouse'].unique().tolist()
            sim = parameter_fit(df1,self.bounds,self.guess)
            output = sim.minimisation()
            data = pd.DataFrame({'mouse':mouse,'alpha':output.x[0],'beta':output.x[1],
                                '- log likelihood':output.fun})
            data_fit = data_fit.append(data)
        return data_fit

In [ ]:
model = parameter_fit(data)
fit = model.data_fit()

In [ ]:
y_axis = fit['alpha'].values.tolist() 
beta = fit['beta'].values.tolist() 
x_axis = ['α'for i in range(43)]
x_axis2 = ['β' for i in range(43)] 

In [ ]:
df2 = pd.DataFrame(list(zip(y_axis, x_axis)),
                columns =['Parameter value', 'Parameter'])

In [ ]:
sns.set_style("whitegrid")
ax = sns.boxplot(x="Parameter", y="Parameter value", data=df2, width = 0.4)
ax = sns.swarmplot(x="Parameter", y="Parameter value", data=df2, color=".25")
ax.set(xlabel=None)
ax.set_ylabel("Parameter value",fontsize=12)
ax.set(ylim=(0, 1.05))
ax.tick_params(labelsize=11)
plt.savefig('boxplotalpha.png', bbox_inches='tight', dpi = 300)

In [ ]:
df3 = pd.DataFrame(list(zip(beta, x_axis2)),
                columns =['Parameter value', 'Parameter'])

In [ ]:
ax = sns.boxplot(x="Parameter", y="Parameter value", data=df3, width = 0.4, color = 'darkorange')
ax = sns.swarmplot(x="Parameter", y="Parameter value", data=df3, color=".25")
ax.set(xlabel=None)
ax.set_ylabel("Parameter value",fontsize=12)
ax.set(ylim=(0, 5.1))
ax.tick_params(labelsize=11)
plt.savefig('boxplotbeta.png', bbox_inches='tight', dpi = 300)